# Fit the RLDDM on individual data

In [2]:
import rlssm
import pandas as pd
import os

## Import the data

In [ ]:
par_path = os.path.abspath(os.path.join(os.getcwd(), os.pardir, os.pardir))
data_path = os.path.join(par_path, 'data/data_experiment.csv')
data = pd.read_csv(data_path, index_col=0)

data = data[data.participant == 10].reset_index(drop=True) # Only select 1 participant

data.head()

## Initialize the model

In [ ]:
# Select model mechanisms:
model = rlssm.RLDDModel(hierarchical_levels=1,
                        separate_learning_rates=False,
                        threshold_modulation=True,
                        nonlinear_mapping=False)

### Fit

In [ ]:
# sampling parameters
n_iter = 1000
n_chains = 2
n_thin = 5

# learning parameters
K = 4 # n options
initial_value_learning = 27.5 # intitial value (Q0)

# bayesian model
alpha_priors = None
drift_scaling_priors = None
threshold_priors = None
ndt_priors = None

In [ ]:
model_fit = model.fit(
    data,
    K,
    initial_value_learning,
    alpha_priors=alpha_priors,
    drift_scaling_priors=drift_scaling_priors,
    threshold_priors=threshold_priors,
    ndt_priors=ndt_priors,
    thin = n_thin,
    iter = n_iter,
    chains = n_chains)

#### get Rhat

In [ ]:
model_fit.rhat.describe()

In [ ]:
model_fit.rhat.head()

#### get wAIC

In [ ]:
model_fit.waic

### Posteriors

In [ ]:
model_fit.samples

In [ ]:
model_fit.trial_samples

In [ ]:
import seaborn as sns
sns.set(context = "talk", 
        style = "white", 
        palette = "husl",
        rc={'figure.figsize':(15, 8)})

In [ ]:
model_fit.plot_posteriors(height=5, show_intervals='HDI');

### Posterior predictives

#### Ungrouped

In [ ]:
pp = model_fit.get_posterior_predictives_df(n_posterior_predictives=100)
pp

In [ ]:
pp_summary = model_fit.get_posterior_predictives_summary(n_posterior_predictives=100)
pp_summary

In [ ]:
model_fit.plot_mean_posterior_predictives(n_posterior_predictives=100, figsize=(20,8), show_intervals='HDI');

In [ ]:
model_fit.plot_quantiles_posterior_predictives(n_posterior_predictives=100, kind='shades');

#### Grouped

In [ ]:
import numpy as np

In [ ]:
data['choice_pair'] = 'AB'
data.loc[(data.cor_option == 3) & (data.inc_option == 1), 'choice_pair'] = 'AC'
data.loc[(data.cor_option == 4) & (data.inc_option == 2), 'choice_pair'] = 'BD'
data.loc[(data.cor_option == 4) & (data.inc_option == 3), 'choice_pair'] = 'CD'

data['block_bins'] = pd.cut(data.trial_block, 8, labels=np.arange(1, 9))

In [ ]:
data.head()

In [ ]:
model_fit.get_grouped_posterior_predictives_summary(
                grouping_vars=['block_label', 'choice_pair'],
                quantiles=[.3, .5, .7],
                n_posterior_predictives=100)

In [ ]:
model_fit.get_grouped_posterior_predictives_summary(
                grouping_vars=['block_bins'],
                quantiles=[.3, .5, .7],
                n_posterior_predictives=100)

In [ ]:
model_fit.plot_mean_grouped_posterior_predictives(grouping_vars=['block_bins'], 
                                                  n_posterior_predictives=100, 
                                                  figsize=(20,8));

In [ ]:
model_fit.plot_quantiles_grouped_posterior_predictives(n_posterior_predictives=100,
                                                        grouping_var='choice_pair',
                                                        kind='shades',
                                                        quantiles=[.1, .3, .5, .7, .9]);

In [ ]:
model_fit.plot_quantiles_grouped_posterior_predictives(
    n_posterior_predictives=300,
    grouping_var='choice_pair',
    palette = sns.color_palette('husl'),
    intervals_kws={'alpha': .3, 'lw':8},
    hue_order=['AB', 'AC', 'BD', 'CD'],
    hue_labels=['ab', 'ac', 'bd', 'cd']);

### Get starting values for further sampling

In [ ]:
sv = model_fit.last_values
sv